# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
gsd_file = "/Users/jimmy/cmelab/data/ITIC/887c520f567aa40dd310a8e1e0c0d6c4/trajectory.gsd"

with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[0]
    end_snap = f[-1]

    

ref_distance = .3563594872561358

start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance

#box = start_snap.configuration.box[:3]
#unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
#unwrap_snap = deepcopy(start_snap)
#unwrap_snap.particles.position = unwrapped_positions
#unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
#comp = from_snapshot(unwrap_snap, scale=0.1)
#comp.visualize().show()



In [3]:
print(start_snap.configuration.box)
print(end_snap.configuration.box)

[70.85166 70.85166 70.85166  0.       0.       0.     ]
[14.170388 14.170388 14.170388  0.        0.        0.      ]


below i have chromo ids. im goin to have it tell me how many atoms per molecule and
take the numbers mudulo that number. this should give the ids for the first molecule

In [4]:
chromo_ids = np.array([69570,69568,69568,69571,69569,69567,69573,69574,69572,69575,69564,69570,69568,69571,69569,69567,69573,69574,69572,69575,69564,69563,69565,69561,69562,69566,69560,69558,69559,69557,69531,69532,69530,69527,69529,69524,69525,69523,69526,69497,69498,69496,69495,69493,69494,69490,69491,69492,69489,69488,69487,69485,69483,69486,69484,69482,69481,69480,69479,69478,69475,69476,69477,69472,69473])


the guts for doin this live in the intramolecular notebook which i guess i didnt push???? come back to it latter

In [5]:
chromo2_ids = chromo_ids +186

In [6]:
aaids=[]

In [7]:
aaids.append(chromo_ids)
aaids.append(chromo2_ids)

In [8]:
print(aaids)

[array([69570, 69568, 69568, 69571, 69569, 69567, 69573, 69574, 69572,
       69575, 69564, 69570, 69568, 69571, 69569, 69567, 69573, 69574,
       69572, 69575, 69564, 69563, 69565, 69561, 69562, 69566, 69560,
       69558, 69559, 69557, 69531, 69532, 69530, 69527, 69529, 69524,
       69525, 69523, 69526, 69497, 69498, 69496, 69495, 69493, 69494,
       69490, 69491, 69492, 69489, 69488, 69487, 69485, 69483, 69486,
       69484, 69482, 69481, 69480, 69479, 69478, 69475, 69476, 69477,
       69472, 69473]), array([69756, 69754, 69754, 69757, 69755, 69753, 69759, 69760, 69758,
       69761, 69750, 69756, 69754, 69757, 69755, 69753, 69759, 69760,
       69758, 69761, 69750, 69749, 69751, 69747, 69748, 69752, 69746,
       69744, 69745, 69743, 69717, 69718, 69716, 69713, 69715, 69710,
       69711, 69709, 69712, 69683, 69684, 69682, 69681, 69679, 69680,
       69676, 69677, 69678, 69675, 69674, 69673, 69671, 69669, 69672,
       69670, 69668, 69667, 69666, 69665, 69664, 69661, 69662, 696

In [9]:
chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, start_snap, aaid, "acceptor", amber_dict))

In [10]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, end_snap, amber_dict, d_cut=100)
print(f"There are {len(qcc_pairs)} chromophore pairs")

There are 1 chromophore pairs


In [12]:
print(chromo_list)

[Chromophore 0 (acceptor): 65 atoms at 2.803 0.400 -3.376, Chromophore 1 (acceptor): 65 atoms at -28.403 22.061 21.011]


In [12]:
i =  0# try any number from 0 to 180
print(f"Pair #{i}:")
visualize_qcc_input(qcc_pairs[i][1])

Pair #0:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")

RuntimeError: Electron number 871 and spin 0 are not consistent
Note mol.spin = 2S = Nalpha - Nbeta, not 2S+1

In [ ]:
%%time
data = eqcc.singles_homolumo(chromo_list, "singles.txt")

In [ ]:
%%time
eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")

In [ ]:
print(chromo_list)

In [ ]:
i = 0
chromo = chromo_list[i]
print(f"Chromophore {i}:")
print(f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} LUMO+1: {chromo.lumo_1:.2f}")
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")